In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import pprint

from hydra import compose, initialize_config_dir

from topollm.config_classes.constants import HYDRA_CONFIGS_BASE_PATH
from topollm.config_classes.main_config import MainConfig
from topollm.logging.initialize_configuration_and_log import initialize_configuration

In [ ]:
# Using hydra in Jupyter notebook:
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
abs_config_dir = pathlib.Path(HYDRA_CONFIGS_BASE_PATH)
with initialize_config_dir(version_base=None, config_dir=str(abs_config_dir)):
    config = compose(
        config_name="main_config",
        overrides=["feature_flags.finetuning.use_wandb=false", "finetuning=finetuning_for_token_classification"],
        return_hydra_config=True,
    )
    print(pprint.pformat(config))

    main_config: MainConfig = initialize_configuration(
        config=config,
    )